In [17]:
class gpt4_tokenizer:
    def __init__(self,text,vocab_size):
        self.text=text
        self.vocab_size=vocab_size
        self.pair_dict = {}
        self.merges = {}


    def get_pair_counts(self,text):
    
        for i in range(len(text) - 1):
            pair = (text[i],text[i+1])
            if pair in self.pair_dict:
                self.pair_dict[pair] += 1
            else:
                self.pair_dict[pair] = 1

        return self.pair_dict 
    

    def pair_switch(token,pair,new_pair):
        i=0
        np=[]
        while i < len(token):
            if i<len(token)-1 and token[i] and token[i]==pair[0] and token[i+1]==pair[1]:
                np.append(new_pair)
                i+=2
            else :
                np.append(token[i])
                i+=1
        return np
    
    
    def train(self,text,vocab_size,verbose=False):
        text = text.encode("utf-8")
        text=list(map(int,text))

        number_of_merges=vocab_size-256
        bpe_tokens=list(text)
        
        for i in range(number_of_merges):
            pair_dict=self.get_pair_counts(bpe_tokens)
            pair = max(pair_dict, key=pair_dict.get)
            if verbose==False:
                continue
            else:
                print(f"Token {pair} is merged as {i+256}")
            bpe_tokens=self.pair_switch(bpe_tokens,pair,i+256)
            self.merges[pair] = i+256
            return self.merges,bpe_tokens
        


    def decode(self,ids):
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]

        tokens = b"".join(vocab[idx] for idx in ids)
        decoded_text = tokens.decode("utf-8", errors="replace")
        return decoded_text
    
    def encoder(self,text):
        # Convert the text to bytes
        tokens=list(text.encode("utf-8"))
        tokens=list(map(int,tokens))
        while len(tokens)>=2:
            pair_counts=self.get_pair_counts(tokens)
            pair=min(pair_counts,key=lambda p:self.merges.get(p,float("inf")))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            tokens=self.merges(tokens,pair,idx)
        return tokens
    


In [18]:
text="""Bonjour! 🌟 Welcome to the magical world of language! In this vast universe of words, where the sky is the limit, let's embark on a journey together, exploring the beauty and diversity of different languages. ¡Hola amigos! ¿Cómo están? Today, we're going to delve into the wonders of multilingualism. Imagine being able to speak fluently in various tongues, connecting with people from all walks of life. C'est fantastique! 🎉

Let's start with English, the language of Shakespeare and modern communication. English is a global language, spoken by millions around the world. From the bustling streets of New York City to the serene countryside of England, English bridges cultures and societies. It's a language of opportunity, innovation, and creativity. So, grab your cup of tea ☕️ and let's dive into the world of English literature and culture.

Pero no podemos olvidar el hermoso idioma español. Con sus ricos sonidos y expresiones poéticas, el español nos lleva en un viaje a través de la pasión y el romance. Desde las vibrantes fiestas de España hasta las playas doradas de América Latina, el español es una celebración de la vida y la diversidad. ¡Viva la lengua española! 🇪🇸

Maintenant, parlons français! Ah, la langue de l'amour et de la sophistication. Le français est un mélange envoûtant de finesse et d'élégance. De Paris, la ville lumière, aux champs de lavande de la Provence, le français évoque un sentiment de joie de vivre. C'est magnifique! 🥖

As we journey through these languages, let's not forget the power of emojis. 😊 Emojis add color and emotion to our digital conversations. Whether it's a smiley face 😄 to brighten someone's day or a heart ❤️ to express love and affection, emojis transcend language barriers and connect us on a deeper level.

Now, let's wrap up our linguistic adventure with a toast 🥂 to the beauty of language! May we continue to explore, learn, and appreciate the rich tapestry of words that make our world a more vibrant and interconnected place. Cheers to language! Salud! 🎊




"""

In [19]:
tokenizer = gpt4_tokenizer(text=text, vocab_size=500)

In [20]:
merges, bpe_tokens = tokenizer.train(text=text, vocab_size=500, verbose=False)

TypeError: cannot unpack non-iterable NoneType object